In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
main_data = pd.read_csv('FIFA2019.csv', index_col=0)

In [2]:
print(f"Rows : {main_data.shape[0]} x Columns : {main_data.shape[1]}")

np.random.seed(1)
main_data.sample(n=10)

Rows : 18207 x Columns : 88


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
18070,236705,E. Dahlqvist,19,https://cdn.sofifa.org/players/4/19/236705.png,Sweden,https://cdn.sofifa.org/flags/46.png,50,64,IFK Göteborg,https://cdn.sofifa.org/teams/2/light/319.png,€60K,€1K,1337,Left,1.0,3.0,2.0,Medium/ Medium,Lean,No,LWB,33.0,"Nov 6, 2016",NaN,2021,5'6,154lbs,39+2,39+2,39+2,43+2,41+2,41+2,41+2,43+2,42+2,42+2,42+2,44+2,42+2,42+2,42+2,44+2,48+2,46+2,46+2,46+2,48+2,48+2,48+2,48+2,48+2,48+2,40.0,26.0,37.0,41.0,22.0,44.0,30.0,26.0,36.0,45.0,62.0,57.0,73.0,48.0,84.0,23.0,92.0,64.0,57.0,25.0,53.0,51.0,43.0,36.0,31.0,52.0,40.0,48.0,51.0,8.0,7.0,12.0,13.0,12.0,€113K
5073,233301,R. Nissen Kristensen,20,https://cdn.sofifa.org/players/4/19/233301.png,Denmark,https://cdn.sofifa.org/flags/13.png,70,79,Ajax,https://cdn.sofifa.org/teams/2/light/245.png,€2.4M,€6K,1855,Right,1.0,3.0,3.0,High/ High,Normal,No,RB,2.0,"Jan 23, 2018",NaN,2022,6'1,154lbs,65+2,65+2,65+2,65+2,65+2,65+2,65+2,65+2,64+2,64+2,64+2,67+2,66+2,66+2,66+2,67+2,69+2,68+2,68+2,68+2,69+2,69+2,67+2,67+2,67+2,69+2,66.0,53.0,65.0,67.0,49.0,65.0,68.0,36.0,66.0,67.0,75.0,76.0,66.0,69.0,59.0,73.0,73.0,87.0,74.0,49.0,70.0,67.0,67.0,63.0,45.0,72.0,66.0,66.0,64.0,10.0,9.0,9.0,6.0,10.0,€3.8M
14368,231064,T. Hudson-Wihongi,23,https://cdn.sofifa.org/players/4/19/231064.png,New Zealand,https://cdn.sofifa.org/flags/198.png,61,67,NaN,https://cdn.sofifa.org/flags/198.png,€0,€0,1613,Right,1.0,3.0,2.0,Medium/ Medium,Normal,No,CDM,21.0,NaN,NaN,NaN,6'1,183lbs,53+2,53+2,53+2,56+2,55+2,55+2,55+2,56+2,57+2,57+2,57+2,58+2,58+2,58+2,58+2,58+2,59+2,60+2,60+2,60+2,59+2,59+2,60+2,60+2,60+2,59+2,53.0,45.0,60.0,62.0,27.0,57.0,46.0,37.0,61.0,60.0,70.0,65.0,62.0,56.0,60.0,41.0,62.0,68.0,70.0,48.0,72.0,58.0,50.0,56.0,52.0,65.0,56.0,58.0,57.0,10.0,6.0,6.0,14.0,8.0,NaN
6757,234022,Adrián Maideira,34,https://cdn.sofifa.org/players/4/19/234022.png,Brazil,https://cdn.sofifa.org/flags/54.png,68,68,Vitória,https://cdn.sofifa.org/teams/2/light/1719.png,€450K,€6K,1753,Left,1.0,3.0,3.0,Medium/ Medium,Normal,No,CAM,10.0,"Jan 1, 2018",NaN,2021,5'7,146lbs,67+2,67+2,67+2,68+2,68+2,68+2,68+2,68+2,67+2,67+2,67+2,66+2,61+2,61+2,61+2,66+2,53+2,50+2,50+2,50+2,53+2,51+2,44+2,44+2,44+2,51+2,55.0,67.0,54.0,70.0,66.0,68.0,65.0,62.0,51.0,63.0,80.0,79.0,82.0,70.0,75.0,70.0,58.0,66.0,58.0,72.0,52.0,33.0,71.0,58.0,68.0,59.0,15.0,41.0,38.0,8.0,8.0,9.0,8.0,13.0,€855K
7829,246056,E. Vega,28,https://cdn.sofifa.org/players/4/19/246056.png,Ecuador,https://cdn.sofifa.org/flags/57.png,67,68,NaN,https://cdn.sofifa.org/flags/57.png,€0,€0,1706,Right,1.0,3.0,3.0,Medium/ Medium,Stocky,No,CDM,17.0,NaN,NaN,NaN,5'6,152lbs,52+2,52+2,52+2,56+2,55+2,55+2,55+2,56+2,57+2,57+2,57+2,60+2,62+2,62+2,62+2,60+2,65+2,66+2,66+2,66+2,65+2,65+2,64+2,64+2,64+2,65+2,55.0,19.0,67.0,70.0,12.0,66.0,48.0,59.0,68.0,66.0,68.0,65.0,63.0,64.0,77.0,66.0,66.0,79.0,64.0,41.0,71.0,68.0,38.0,55.0,60.0,54.0,59.0,64.0,62.0,11.0,14.0,9.0,6.0,6.0,NaN
8497,202179,Z. Ismail,24,https://cdn.sofifa.org/players/4/19/202179.png,England,https://cdn.sofifa.org/flags/14.png,67,70,Walsall,https://cdn.sofifa.org/teams/2/light/1803.png,€975K,€3K,1688,Right,1.0,5.0,4.0,High/ Medium,Normal,No,RM,10.0,"Jul 1, 2018",NaN,2019,5'10,165lbs,62+2,62+2,62+2,66+2,64+2,64+2,64+2,66+2,64+2,64+2,64+2,66+2,59+2,59+2,59+2,66+2,53+2,48+2,48+2,48+2,53+2,50+2,41+2,41+2,41+2,50+2,57.0,60.0,45.0

In [3]:
main_data.drop(['ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Jersey Number'], axis=1, inplace=True)

In [5]:
nulls_series             = main_data.isna().sum()         # Get a series counting number of empty values for each column
nonnulls_series          = main_data.notnull().sum()      # Get a series counting number of non empty valuesfor each column 
nulls_percentage         = ((nulls_series * 100)/(nulls_series + nonnulls_series)).astype(float)
column_datatypes         = main_data.dtypes               # Get a series containing data types for each column 

nulls_count_df = pd.concat(
    [nulls_series, nonnulls_series, nulls_percentage, column_datatypes],
    axis=1,
    keys=["Nulls", "Non-Nulls","Nulls %", "Type"], 
    sort=True)
cm = sns.light_palette("red", as_cmap=True)

nulls_count_df.style.background_gradient(cmap=cm, subset=pd.IndexSlice[:, ['Nulls %']]).format(formatter={('Nulls %'): "{:.2f}%"})

,Nulls,Non-Nulls,Nulls %,Type
Acceleration,48,18159,0.26%,float64
Age,0,18207,0.00%,int64
Aggression,48,18159,0.26%,float64
Agility,48,18159,0.26%,float64
Balance,48,18159,0.26%,float64
BallControl,48,18159,0.26%,float64
Body Type,48,18159,0.26%,object
CAM,2085,16122,11.45%,object
CB,2085,16122,11.45%,object
CDM,2085,16122,11.45%,object


In [13]:
money_cols = []
for col_name in main_data.columns[main_data.dtypes == "object"]:
    if main_data[col_name].str.startswith('€').any():
        money_cols.append(col_name)
        
print(money_cols)

['Value', 'Wage', 'Release Clause']


In [16]:
def income_to_num(income_val):
    if isinstance(income_val, str):
        multiplier = 1
        if income_val.endswith('K'):
            multiplier = 1000
        elif income_val.endswith('M'):
            multiplier = 1000000
        return float(income_val.replace('€', '').replace('K', '').replace('M', '')) * multiplier
    else:
        return np.nan
    
for col_name in money_cols:
    main_data[col_name] = main_data[col_name].apply(income_to_num)
    
main_data[money_cols].head()

,Value,Wage,Release Clause
0,110500000.0,565000.0,226500000.0
1,77000000.0,405000.0,127100000.0
2,118500000.0,290000.0,228100000.0
3,72000000.0,260000.0,138600000.0
4,102000000.0,355000.0,196400000.0


In [22]:
def position_to_num(pos_val):
    if isinstance(pos_val, str):
        return float(pos_val.split('+')[0])
    else:
        return np.nan
    
position_cols = ['LS', 'ST', 'RS', 'LW', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM','LCM', 'CM', 'RCM']

for col_name in position_cols:
    main_data[col_name] = main_data[col_name].apply(position_to_num)
    
main_data[position_cols].head(200)

,LS,ST,RS,LW,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
